# Getting Started

Go to the setupTable and input your unique token

DO NOT USE MY KEY (Blake)

In [1]:
import requests
import time
import json

In [2]:
class Stack():
    def __init__(self):
        self.stack = []
    def push(self, value):
        self.stack.append(value)
    def pop(self):
        if self.size() > 0:
            return self.stack.pop()
        else:
            return None
    def size(self):
        return len(self.stack)

In [3]:
class Queue():
    def __init__(self):
        self.queue = []
    def enqueue(self, value):
        self.queue.append(value)
    def dequeue(self):
        if self.size() > 0:
            return self.queue.pop(0)
        else:
            return None
    def size(self):
        return len(self.queue)

In [4]:
# Import map and room info
with open('worldMap.json', 'r') as f:
    worldMap = json.load(f)
    
with open('worldRoomIndex.json', 'r') as j:
    worldRoomIndex = json.load(j)

In [5]:
# Utils
inverseMap = {'n':'s', 's':'n', 'e':'w', 'w':'e'}

In [6]:
setupTable = {
    "url": 'https://lambda-treasure-hunt.herokuapp.com/api/',
    "token": "",
    "init": 'adv/init/',
    "move": 'adv/move/',
    "take": 'adv/take/',
    "status": 'adv/status/',
    "examine": 'adv/examine/',
    "mine": 'bc/mine/',
    "proof": 'bc/last_proof/',
}
header={'Authorization': 'Token ' + setupTable['token']}
init = setupTable['url'] + setupTable['init']
move = setupTable['url'] + setupTable['move']
take = setupTable['url'] + setupTable['take']
status = setupTable['url'] + setupTable['status']
examine = setupTable['url'] + setupTable['examine']
mine = setupTable['url'] + setupTable['mine']
proof = setupTable['url'] + setupTable['proof']

In [7]:
class Player:
    
    def __init__(self):
        self.initResponse = requests.get(url=init, headers=header)
        self.playersMap = worldMap
        self.coolDown = None
        self.currentRoomID = None
        self.previousRoomID = None
        self.lastMoveDirection = None
        self.playerItems = []
        
        self.currentRoomInfo = None
        
    def getRoomInfo(self):
        time.sleep(2)
        response = requests.get(url=init, headers=header)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(response.json())
            print(f'400 status code at getRoomInfo method {response.status_code}')
            
    def assignCurrentRoomInfo(self):
        self.currentRoomInfo = self.getRoomInfo()
        
    def assignCurrentRoomID(self):
        self.currentRoomID = self.getCurrentRoomID()
            
    def getCurrentRoomID(self):
        return self.currentRoomInfo['room_id']
            
    def writeNewRoomToMap(self):
        exits = self.currentRoomInfo['exits']
        
        if self.currentRoomID not in self.playersMap:
            self.playersMap[self.currentRoomID] = {key: '?' for key in exits}
            
    def writeMoveDirectionsForCurrentAndPreviousRoom(self, previousRoomID, currentRoomID, direction):
        self.playersMap[previousRoomID][direction] = self.currentRoomID
        self.playersMap[currentRoomID][inverseMap[direction]] = self.previousRoomID
        
    def moveRoom(self, direction):
        if str(self.currentRoomID) in self.playersMap and self.playersMap[str(self.currentRoomID)][direction] !='?':
            print("Wise Explore")
            
            self.previousRoomID = self.currentRoomID
            
            nextRoom = self.playersMap[str(self.currentRoomID)][direction]
            
            postRequestBody = {"direction":direction, "next_room_id":str(nextRoom)}
            response = requests.post(url=move, headers=header, json=postRequestBody)
            self.currentRoomInfo = response.json()
            self.coolDown = response.json()['cooldown']
            
            time.sleep(self.coolDown)
            self.currentRoomID = self.getCurrentRoomID()
            self.lastMoveDirection = direction
            
        else:
            print('Normal Explore')
            self.previousRoomID = self.currentRoomID
            
            postRequestBody = {"direction":direction}
            response = requests.post(url=move, headers=header, json=postRequestBody)

            self.currentRoomInfo = response.json()
            self.currentRoomID = self.getCurrentRoomID()
            
            if response.status_code == 200:
                self.coolDown = response.json()['cooldown']
                time.sleep(self.coolDown)
                
                self.lastMoveDirection = direction
                
                self.writeNewRoomToMap()
                self.writeMoveDirectionsForCurrentAndPreviousRoom(self.previousRoomID, self.currentRoomID, direction)
            else:
                print("The move was not a success")
                
        return self.getRoomInfo()
    
    def findUnexploredRoom(self):
        for direction in self.playersMap[self.currentRoomID]:
            if self.playersMap[self.currentRoomID][direction] == '?':
                return direction
        return direction
    
    def takeItem(self, itemName):
        postRequestBody = {'name': itemName}
        response = requests.post(url=move, headers=header, json=postRequestBody)
        self.coolDown = response.json()['cooldown']
        
        self.playerItems.append(itemName)
        print("You have picked up ", itemName)
        
    def dropItem(self, itemName):
        pass
        
    def checkInventory(self):
        print(self.playerItems)
        
    def status(self):
        response = requests.post(url=status,headers=header)
        self.coolDown = response.json()['cooldown']
        return response.json()
    
    def examineWell(self):
        postRequestBody = {"name":"WELL"}
        response = requests.post(url=examine, headers=header, json=postRequestBody)
        self.coolDown = response.json()['cooldown']
        return response.json()
    
    def mineWell(self, newProof):
        postRequestBody = {"proof":newProof}
        response = requests.post(url=mine, headers=header, json=postRequestBody)
        self.coolDown = response.json()['cooldown']
        return response.json()
    
    def lastProof(self):
        response = requests.get(url=proof, headers=header)
        self.coolDown = response.json()['cooldown']
        return response.json()
    

In [8]:
def exploreWorld(player):
    print('Initializing...')
    
    roomStack = Stack()
    moveStack = Stack()
    count = 0
    

    player.currentRoomInfo = player.getRoomInfo()
    print('Loading currentRoomInfo')
    time.sleep(5)
    
    player.currentRoomID = player.currentRoomInfo['room_id']
    player.writeNewRoomToMap()
    
    roomStack.push(player.currentRoomID)
    
    roomDict = {}
    roomDict[player.currentRoomID] = player.currentRoomInfo
    
    while roomStack.size() > 0:
        
        direction = player.findUnexploredRoom()
        print("Direction", direction)
        
        if player.playersMap[player.currentRoomID][direction] == '?':
            
            
            player.moveRoom(direction)
            moveStack.push(direction)
            
            print("Leaving room: "+str(player.previousRoomID)+" entering room: "+str(player.currentRoomID))
            
            roomStack.push(player.currentRoomID)
            
            if player.currentRoomID not in roomDict:
                roomDict[player.currentRoomID] = player.currentRoomInfo
                
        else:
            print("roomStack:", roomStack.stack)
            print('moveStack:', moveStack.stack)
            roomStack.pop()
            lastMove = moveStack.stack[-1]
            backTrackingDirection = inverseMap[lastMove]
            
            if roomStack.size() > 0:
                player.previousRoomID = player.currentRoomID
                player.moveRoom(backTrackingDirection)
                player.lastMoveDirection = moveStack.stack[-1]      

            moveStack.pop()

    return roomDict          

In [9]:
def bfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathQueue = Queue()
    directionQueue = Queue()
    
    pathQueue.enqueue([str(starting_vertex)])
    directionQueue.enqueue(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathQueue.dequeue()
        directionPath = directionQueue.dequeue()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathQueue.enqueue(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionQueue.enqueue(directionCopy)

    return None

In [10]:
def dfs(player, starting_vertex, destination_vertex):
    """
    Return a list containing the shortest path from
    starting_vertex to destination_vertex in
    breath-first order.
    """

    # Create an empty queue and enqueue the starting vertex ID
    pathStack = Stack()
    directionStack = Stack()
    
    pathStack.push([str(starting_vertex)])
    directionStack.push(['Starting Location'])

    # Create a Set to store visited vertices
    visited = set()

    # While the queue is not empty...
    while destination_vertex not in visited:
        # Dequeue the first vertex
        path = pathStack.pop()
        directionPath = directionStack.pop()

        # If that vertex has not been visited
        v = path[-1]
        
        if v not in visited:
            if v == destination_vertex:
                return path, directionPath
            visited.add(v)
            
        for direction, neighboringRoom in player.playersMap[v].items():
            path_copy = list(path)
            path_copy.append(str(neighboringRoom))
            pathStack.push(path_copy)
            
            directionCopy = list(directionPath)
            directionCopy.append(direction)
            directionStack.push(directionCopy)

    return None

In [11]:
def moveToSpecificRoom(player, starting_vertex, destination_vertex):
    path, directionPath = bfs(player, starting_vertex, destination_vertex)
    
    print(path)
    directions = directionPath[1:]
    print(directions)
    for direction in directions:
        player.moveRoom(direction)
        time.sleep(player.coolDown)
        
        print("You have reached your destination")

In [12]:
player = Player()
player.assignCurrentRoomInfo()
player.assignCurrentRoomID()

In [13]:
player.currentRoomID

55

In [79]:
moveToSpecificRoom(player, player.currentRoomID, '206')

['39', '53', '95', '119', '134', '147', '200', '206']
['n', 'n', 'n', 'n', 'n', 'n', 'e']
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination


In [80]:
player.examineWell()

{'room_id': 206,
 'title': 'A misty room',
 'description': 'You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction.',
 'coordinates': '(64,70)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': [],
 'items': ['tiny treasure'],
 'exits': ['n', 'e', 'w'],
 'cooldown': 12.5,
 'errors': ['Item not found: +5s CD'],
 'messages': []}

In [65]:
player.lastProof()

{'cooldown': 26.204508, 'errors': ['Cooldown Violation: +5s CD']}

In [206]:
lastProof

{'proof': 13756334,
 'difficulty': 6,
 'cooldown': 1.0,
 'messages': [],
 'errors': []}

In [83]:
import hashlib

def validProof(lastProof, newProof):

    # Is a string of 0s based on the difficulty level
    numOfZeros = ''.join(['0']*leadingZeros)

    # Return True if proof is successful
    guess = f'{lastProof}{newProof}'.encode()

    guessHash = hashlib.sha256(guess).hexdigest()

    return guessHash[:leadingZeros] == numOfZeros

def proofWork(lastProof):

    # Turn it to a string
    # lastProofString = json.dumps(lastProof, sort_keys=True)

    # Increment proof by 1 each time it is False
    newProof = 0
    # Guess and check
    while validProof(lastProof, newProof) is False:
        newProof += 1
    return newProof

#--------------------------------------------#

block = player.lastProof()

if block['proof']:
    # How many leading zeros we need
    leadingZeros = block['difficulty']

    # Get last proof number
    lastProof = block['proof']
    print('Last Proof:', lastProof, '\n')

    proofNum = proofWork(lastProof)
else:
    block

Last Proof: 38853805 



In [75]:
chr(0b01001000)

'H'

In [84]:
player.lastProof()['proof']

38853805

In [85]:
player.mineWell(proofNum)

{'index': 4922,
 'transactions': 'adv_blockchain.Transaction.None',
 'proof': 3410411,
 'previous_hash': 'ec12c2d23e37d7e557153242c9c860aebfdf828bc9db2a1a19197426baf1d459',
 'cooldown': 15.0,
 'messages': ['New Block Forged'],
 'errors': []}

In [79]:
player.getRoomInfo()

{'room_id': 55,
 'title': 'Wishing Well',
 'description': "You are standing besides a large well. A sign next the well reads 'EXAMINE WELL, FIND WEALTH'.",
 'coordinates': '(63,61)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': ['ken_ridenour',
  'Micah',
  '\n',
  'Mario Greenbeard',
  'Labyrinth65',
  'NP',
  'Dr Cooldown'],
 'items': [],
 'exits': ['w'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}

In [86]:
player.status()

{'cooldown': 7.49178, 'errors': ['Cooldown Violation: +5s CD']}

In [39]:
player.moveRoom('e')

Wise Explore


{'room_id': 55,
 'title': 'Wishing Well',
 'description': "You are standing besides a large well. A sign next the well reads 'EXAMINE WELL, FIND WEALTH'.",
 'coordinates': '(63,61)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': ['ken_ridenour',
  'Micah',
  '\n',
  'Mario Greenbeard',
  'Labyrinth65',
  'NP',
  'Dr Cooldown'],
 'items': [],
 'exits': ['w'],
 'cooldown': 1.0,
 'errors': [],
 'messages': []}

In [48]:
moveToSpecificRoom(player, player.currentRoomID, '55')

['22', '18', '12', '9', '3', '2', '0', '4', '23', '26', '55']
['n', 'n', 'n', 'n', 'w', 'n', 'e', 'n', 'e', 'e']
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination
Wise Explore
You have reached your destination


In [69]:
worldRoomIndex['55']

{'room_id': 55,
 'title': 'Wishing Well',
 'description': "You are standing besides a large well. A sign next the well reads 'EXAMINE WELL, FIND WEALTH'.",
 'coordinates': '(63,61)',
 'elevation': 0,
 'terrain': 'NORMAL',
 'players': [],
 'items': ['small treasure'],
 'exits': ['w'],
 'cooldown': 15.0,
 'errors': [],
 'messages': ['You have walked east.']}

In [67]:
for k,v in worldRoomIndex.items():
    for j,l in v.items():
        if j == 'description':
            print(k, j, l, '\n')

388 description You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction. 

477 description You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction. 

483 description You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction. 

382 description You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction. 

322 description You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction. 

435 description You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction. 

261 description You are standing on grass and surrounded by a dense mist. You can barely make out the exits in any direction. 

277 description You are standing on grass and surrounded by a dense mist. You can barely make out the ex